TF serving demo:
* train a keras model with multiple outputs
* serve the model
    * compare timing with in-memory inference

In [1]:
# Resources for blog post
# http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/
# https://github.com/keras-team/keras/blob/master/examples/mnist_dataset_api.py

In [2]:
# 1. Store a text file for train generator

In [3]:
IM_SIZE=128

In [4]:
import os
import numpy as np
import pandas as pd

def flatten(l): return [item for sublist in l for item in sublist]

In [5]:
# Read dataframe
df_train = pd.read_csv("./data/train_v2.csv")

# Make label maps
labels = sorted(list(set(flatten([l.split(' ') for l in df_train['tags'].values]))))

weather_labels = ['clear', 'cloudy', 'haze', 'partly_cloudy']
ground_labels = [l for l in labels if l not in weather_labels]

label_map = {l:i for i, l in enumerate(labels)}
wlabel_map = {l: i for i, l in enumerate(weather_labels)}
glabel_map = {l: i for i, l in enumerate(ground_labels)}

def get_labels_binary(s, labelmap):
    labels = np.zeros(len(labelmap), dtype=np.int64)
    idx = [v for v in [labelmap[w] for w in s.split(' ')]]
    labels[idx] = 1
    return labels

def array_to_str(arr):
    return(str(arr.tolist()))

df_train['label'] = df_train['tags'].apply(get_labels_binary, args=(label_map,))

In [6]:
# Map everything to strings
df_train['label']  = df_train['label'].map(array_to_str)

In [7]:
# Save as text file
df_train.drop('tags', axis=1).to_csv('./data/TRAIN_kaggle.csv', index=None)

In [8]:
pd.read_csv('./data/TRAIN_kaggle.csv').head()

,image_name,label
0,train_0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
1,train_1,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,train_2,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,train_3,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,train_4,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, ..."


In [9]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import ast 
import random

def randomHorizontalFlip(image, p=0.5):
    """Do a random horizontal flip with probability p"""
    if np.random.random() < p:
        image = np.fliplr(image)
    return image


def randomVerticalFlip(image, p=0.5):
    """Do a random vertical flip with probability p"""
    if np.random.random() < p:
        image = np.flipud(image)
    return image

Using TensorFlow backend.


In [10]:
import random
import ast
from keras.utils import Sequence

class KagglePlanetSequence(Sequence):

    def __init__(self, file_path, data_path, im_size, batch_size, shuffle, mode='train'):
        self.df = pd.read_csv(file_path)
        self.dp = data_path
        self.bsz, self.imsz = batch_size, im_size
        self.shuf = shuffle
        self.mode = mode
        
        # List of image paths, np array of labels
        self.im_list = [os.path.join(self.dp, v + '.jpg') for v in self.df['image_name'].tolist()]
        self.lab_arr = np.array([ast.literal_eval(l) for l in self.df['label']])
    
    def __len__(self):
        return int(np.ceil(len(self.df) // self.bsz))

    def on_epoch_end(self):
        # Shuffles indexes after each epoch
        self.indexes = range(len(self.im_list)) 
        if self.shuf:
            self.indexes = random.sample(self.indexes, k=len(self.indexes))
            
    def get_batch_features(self, idx):
        fnames = self.im_list[idx*self.bsz:(idx+1)*self.bsz]
        return np.array([img_to_array(load_img(f, target_size=(self.imsz, self.imsz))) / 255. for f in fnames])
    
    def get_batch_labels(self, idx):
        if self.mode == 'test':
            return None
        return self.lab_arr[idx*self.bsz:(idx+1)*self.bsz]
    
    def __getitem__(self, idx):
        batch_x = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_x, batch_y

In [11]:
# 3. Train a model
import tensorflow as tf
import numpy as np
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Layer, Concatenate, Add, Subtract
from keras.layers import BatchNormalization, Dropout, Activation
from keras.layers import MaxPooling2D
from keras.layers import Conv2D, Conv2DTranspose, Reshape, Multiply, Dot
from keras.layers import BatchNormalization
from keras.engine.topology import Layer
from keras.models import Model
from keras.applications import VGG16, ResNet50
import keras.backend as K
from keras import metrics
        
class CNN_classifier(object):

    def __init__(self, im_size,  n_labels):
        """
        CNN for multi-label image classification with binary relevance
        """
        
        self.im_size = im_size
        self.n_labels = n_labels
        self.dropout_rate = 0.15
        self.n_neurons = 128  # Number of neurons in dense layers
        # build model on init
        self.build()

    def build(self):
        # Define input
        self.x = Input(shape=(self.im_size, self.im_size, 3))

        # Convolutional layers
        conv_1 = Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(self.x)
        conv_1 = MaxPooling2D(padding='same')(conv_1)
        conv_2 = Conv2D(32, kernel_size=(3, 3),
                        padding='same', activation='relu')(conv_1)
        conv_2 = MaxPooling2D(padding='same')(conv_2)

        # Flatten
        conv_flat = Flatten()(conv_2)
        # Fully connected layers
        fc_1 = Dense(self.n_neurons, activation='relu')(conv_flat)
        fc_1 = Dropout(self.dropout_rate)(fc_1)
        fc_2 = Dense(self.n_neurons, activation='relu')(fc_1)
        self.fc_2 = Dropout(self.dropout_rate)(fc_2)

        # Output layers: n_classes output nodes for binary relevance
        self.y = Dense(self.n_labels, activation='sigmoid')(self.fc_2)

        self.model = Model(inputs=self.x, outputs=self.y)

In [12]:
batch_size = 32
seq = KagglePlanetSequence('./data/TRAIN_kaggle.csv', './data/train', im_size=IM_SIZE, batch_size=batch_size, shuffle=False)

In [13]:
# Store model for TF serving
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

export_path = './models/kaggleplanet/1'

model = CNN_classifier(IM_SIZE, 17).model
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit_generator(generator=seq, verbose=1, epochs=1, use_multiprocessing=True, workers=4)

Epoch 1/1
1264/1264 [==============================] - 47s 37ms/step - loss: 0.2160


In [14]:
builder = saved_model_builder.SavedModelBuilder(export_path)
    
with K.get_session() as sess:
    #K.set_session(sess)
    K.set_learning_phase(0)
    
    signature = predict_signature_def(inputs={'images': model.input},
                              outputs={'labels': model.output})
    builder.add_meta_graph_and_variables(sess=sess,
                                 tags=[tag_constants.SERVING],
                                 signature_def_map={'predict': 
                                                   signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


FailedPreconditionError: Attempting to use uninitialized value conv2d_1/kernel
	 [[Node: conv2d_1/kernel/_12 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_16_conv2d_1/kernel", _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_1/kernel)]]
	 [[Node: Adam/decay/_5 = _Recv[_start_time=0, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_12_Adam/decay", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](^save/ShardedFilename, ^save/SaveV2/tensor_names, ^save/SaveV2/shape_and_slices)]]

In [ ]:
test_seq = KagglePlanetSequence('./data/TRAIN_kaggle.csv', './data/train', im_size=IM_SIZE, batch_size=batch_size, shuffle=False, mode='test')
predictions = model.predict_generator(generator=test_seq, verbose=1, use_multiprocessing=True, workers=4)

In [ ]:
# BONUS: Compare training speed with Tf records

In [ ]:
# Serialize images, together with labels, to TF records
from tqdm import tqdm_notebook as tqdm 
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

tf_records_filename = './data/KagglePlanetTFRecord_{}'.format(IM_SIZE)
writer = tf.python_io.TFRecordWriter(tf_records_filename)

# List of image paths, np array of labels
im_list = [os.path.join('./data/train', v + '.jpg') for v in df_train['image_name'].tolist()]
lab_arr = np.array([ast.literal_eval(l) for l in df_train['label']])

for i in tqdm(range(len(df_train))):
    labels = lab_arr[i].astype(np.float32)
    im = np.array(img_to_array(load_img(im_list[i], target_size=(IM_SIZE, IM_SIZE))) / 255.)
    lab_raw = labels.tostring()
    im_raw = im.tostring()
    
    example = tf.train.Example(features=tf.train.Features(feature={'image': _bytes_feature(im_raw),
                                                                  'labels': _bytes_feature(lab_raw)}))
    
    writer.write(example.SerializeToString())
    
writer.close

In [ ]:
from tensorflow import FixedLenFeature
featdef = {'image': FixedLenFeature(shape=[], dtype=tf.string),
          'labels': FixedLenFeature(shape=[], dtype=tf.string)
          }

In [ ]:
def _parse_record(example_proto, clip=False):
    ex = tf.parse_single_example(example_proto, featdef)
    
    im = tf.decode_raw(ex['image'], tf.float32)
    im = tf.reshape(im, (IM_SIZE, IM_SIZE, 3))
    lab = tf.decode_raw(ex['labels'], tf.float32)
    return im, lab

# Construct a dataset iterator
ds_train = tf.data.TFRecordDataset('./data/KagglePlanetTFRecord_{}'.format(IM_SIZE)).map(_parse_record).batch(batch_size)
iterator = tf.data.Iterator.from_structure(ds_train.output_types, ds_train.output_shapes)

ds_tr_init = iterator.make_initializer(ds_train)

x,y = iterator.get_next()

In [ ]:
# Train the model
steps_per_epoch = len(df_train) // batch_size
with tf.Session() as sess:
    K.set_session(sess)
    sess.run(ds_tr_init)
   
    # Rewire network to tie it into the generator
    # Define input
    inp = Input(tensor=x)

    # Convolutional layers
    conv_1 = Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(x)
    conv_1 = MaxPooling2D(padding='same')(conv_1)
    conv_2 = Conv2D(32, kernel_size=(3, 3),
                padding='same', activation='relu')(conv_1)
    conv_2 = MaxPooling2D(padding='same')(conv_2)

    # Flatten
    conv_flat = Flatten()(conv_2)
    # Fully connected layers
    fc_1 = Dense(128, activation='relu')(conv_flat)
    fc_1 = Dropout(0.15)(fc_1)
    fc_2 = Dense(128, activation='relu')(fc_1)
    fc_2 = Dropout(0.15)(fc_2)

    # Output layers: n_classes output nodes for binary relevance
    output = Dense(17, activation='sigmoid')(fc_2)

    model = Model(inputs=inp, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', target_tensors=[y])
    print(model.summary())
    steps_per_epoch = len(df_train) // batch_size
    model.fit(steps_per_epoch=steps_per_epoch, verbose=1, epochs=1)